In [1]:
import patchmatch as pm
import imageio

import numpy as np

/usr/local/Cellar/python/3.7.2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'patchmatch' does not match runtime version 3.7
  return f(*args, **kwds)


In [18]:
def patch_match(source_img, target_img):
    height, width, _ = source_img.shape
    offsets = np.empty((height, width, 2), dtype=np.float32)
    costs = np.empty((height, width), dtype=np.float32)

    PATCHMATCH_SIZE = 8
    MAX_OFFSET = 100
    MIN_OFFSET = 10
    pm.pm(source_img.astype(np.float32),
          target_img.astype(np.float32),
          offsets,
          costs,
          PATCHMATCH_SIZE,
          MIN_OFFSET,
          MAX_OFFSET)

    return offsets[:, :, ::-1]

In [19]:
imgs = []
img = imageio.imread('bike_a.png')
img = (img / 255).astype(np.float64)
imgs.append(img)
img = imageio.imread('bike_b.png')
img = (img / 255).astype(np.float64)
imgs.append(img)
# lib_offsets = patch_match(imgs[0], imgs[1])
# np.save('lib_offset.npy', lib_offsets)
lib_offsets = np.load('lib_offset.npy')

In [20]:
import os

import pyopencl as cl

os.environ['PYOPENCL_NO_CACHE'] = '1'
os.environ['PYOPENCL_NO_CACHE'] = '1'
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

filename = 'patchmatch.c'
PATCH_SIZE = 8


HEIGHT, WIDTH, _ = img.shape
ctx = cl.create_some_context(False)
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
program = cl.Program(ctx, open(filename).read()).build()

# NNF_HEIGHT = HEIGHT - (HEIGHT % PATCH_SIZE)
# NNF_WIDTH = WIDTH - (WIDTH % PATCH_SIZE)
NNF_HEIGHT = HEIGHT - PATCH_SIZE
NNF_WIDTH = WIDTH - PATCH_SIZE
nnf = np.empty((NNF_HEIGHT, NNF_WIDTH, 3))

In [21]:
inputBuf = [cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=imgs[i]) for i in range(2)]
outputBuf = cl.Buffer(ctx, mf.WRITE_ONLY | mf.COPY_HOST_PTR, nnf.nbytes, hostbuf=nnf)

GLOBAL_SIZE = (NNF_HEIGHT, NNF_WIDTH)
program.randomfill(queue, GLOBAL_SIZE, None,
                                np.int32(PATCH_SIZE),  # patchHeight
                                np.int32(PATCH_SIZE),  # patchWidth
                                np.int32(HEIGHT),  # height
                                np.int32(WIDTH),  # width
                                inputBuf[0],
                                inputBuf[1],
                                outputBuf)


init_nnf = np.empty_like(nnf)
cl.enqueue_copy(queue, init_nnf, outputBuf)
nnf = np.copy(init_nnf)

In [22]:
# mf = cl.mem_flags

for i in range(1):
    mf = cl.mem_flags
    inputBuf = [cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=imgs[i]) for i in range(2)]
    outputBuf = cl.Buffer(ctx, mf.READ_WRITE | mf.COPY_HOST_PTR, nnf.nbytes, hostbuf=nnf)

    ITER = i + 1
    program.propagate(queue, GLOBAL_SIZE, None,
                                   np.int32(PATCH_SIZE),  # patchHeight
                                   np.int32(PATCH_SIZE),  # patchWidth
                                   np.int32(HEIGHT),  # height
                                   np.int32(WIDTH),  # width
                                   np.int32(ITER),
                                   inputBuf[0],
                                   inputBuf[1],
                                   outputBuf)


    output_nnf = np.empty_like(nnf)
    cl.enqueue_copy(queue, output_nnf, outputBuf)
    nnf = output_nnf.copy()

In [23]:
nnf.astype(np.int)

array([[[ 10,   2,   3],
        [ 10,   3,   2],
        [  9,   8,   0],
        ...,
        [ 19, 210,   0],
        [ 19, 211,   0],
        [ 19, 206,   0]],

       [[  7,  10,   0],
        [  7,  11,   0],
        [  7,  12,   0],
        ...,
        [ 20, 210,   0],
        [ 20, 211,   0],
        [ 20, 206,   0]],

       [[  8,  10,   0],
        [  8,  11,   0],
        [  8,  12,   0],
        ...,
        [ 21, 194,   0],
        [ 21, 195,   0],
        [ 21, 206,   0]],

       ...,

       [[153,   2,   0],
        [163,  14,   0],
        [164,  12,   0],
        ...,
        [ 93, 213,   0],
        [ 93, 214,   0],
        [ 93, 215,   0]],

       [[145,  11,   0],
        [164,  14,   0],
        [165,  12,   0],
        ...,
        [ 94, 213,   0],
        [ 94, 214,   0],
        [ 94, 215,   0]],

       [[146,  11,   0],
        [165,  14,   0],
        [166,  12,   0],
        ...,
        [ 95, 213,   0],
        [ 95, 214,   0],
        [ 95, 215,   0]]

In [24]:
tmp = nnf.copy()
tmp

array([[[1.00000000e+01, 2.00000000e+00, 3.03730873e+00],
        [1.00000000e+01, 3.00000000e+00, 2.00692042e+00],
        [9.00000000e+00, 8.00000000e+00, 2.00445982e-01],
        ...,
        [1.90000000e+01, 2.10000000e+02, 3.12341407e-02],
        [1.90000000e+01, 2.11000000e+02, 3.30180700e-02],
        [1.90000000e+01, 2.06000000e+02, 2.32218378e-02]],

       [[7.00000000e+00, 1.00000000e+01, 1.62106882e-01],
        [7.00000000e+00, 1.10000000e+01, 2.01768551e-02],
        [7.00000000e+00, 1.20000000e+01, 1.12402922e-01],
        ...,
        [2.00000000e+01, 2.10000000e+02, 3.89542484e-02],
        [2.00000000e+01, 2.11000000e+02, 4.03383314e-02],
        [2.00000000e+01, 2.06000000e+02, 2.87889273e-02]],

       [[8.00000000e+00, 1.00000000e+01, 1.68289120e-01],
        [8.00000000e+00, 1.10000000e+01, 3.35409458e-02],
        [8.00000000e+00, 1.20000000e+01, 3.46251442e-01],
        ...,
        [2.10000000e+01, 1.94000000e+02, 6.42983468e-02],
        [2.10000000e+01, 1.95

In [25]:
nnf.astype(np.int)

array([[[ 10,   2,   3],
        [ 10,   3,   2],
        [  9,   8,   0],
        ...,
        [ 19, 210,   0],
        [ 19, 211,   0],
        [ 19, 206,   0]],

       [[  7,  10,   0],
        [  7,  11,   0],
        [  7,  12,   0],
        ...,
        [ 20, 210,   0],
        [ 20, 211,   0],
        [ 20, 206,   0]],

       [[  8,  10,   0],
        [  8,  11,   0],
        [  8,  12,   0],
        ...,
        [ 21, 194,   0],
        [ 21, 195,   0],
        [ 21, 206,   0]],

       ...,

       [[153,   2,   0],
        [163,  14,   0],
        [164,  12,   0],
        ...,
        [ 93, 213,   0],
        [ 93, 214,   0],
        [ 93, 215,   0]],

       [[145,  11,   0],
        [164,  14,   0],
        [165,  12,   0],
        ...,
        [ 94, 213,   0],
        [ 94, 214,   0],
        [ 94, 215,   0]],

       [[146,  11,   0],
        [165,  14,   0],
        [166,  12,   0],
        ...,
        [ 95, 213,   0],
        [ 95, 214,   0],
        [ 95, 215,   0]]

In [26]:
grids = np.meshgrid(np.arange(NNF_WIDTH), np.arange(NNF_HEIGHT), indexing="xy")
cord = np.concatenate((grids[1][..., np.newaxis], grids[0][..., np.newaxis]), axis=2)

In [40]:
a = nnf[:, :, :2] - cord

In [41]:
lib_offsets.astype(np.int)[:NNF_HEIGHT, :NNF_WIDTH]

array([[[  8,   6],
        [ 12,   7],
        [ 12,   7],
        ...,
        [  2,  10],
        [ 24, -35],
        [ 24, -35]],

       [[  8,   6],
        [ 12,   7],
        [ 12,   7],
        ...,
        [  2,  10],
        [ 24, -35],
        [ 24, -35]],

       [[  8,   6],
        [  8,   6],
        [ 12,   7],
        ...,
        [  2,  10],
        [ 24, -35],
        [ 24, -35]],

       ...,

       [[-21,   0],
        [-21,   0],
        [-21,   0],
        ...,
        [  9,   6],
        [  9,   6],
        [  9,   6]],

       [[-21,   0],
        [-21,   0],
        [-21,   0],
        ...,
        [  9,   6],
        [  9,   6],
        [  9,   6]],

       [[-21,   0],
        [-21,   0],
        [-21,   0],
        ...,
        [  5,   9],
        [  5,   9],
        [-63,   0]]])

In [42]:
np.where(a == -217)

(array([159]), array([229]), array([1]))

In [43]:
NNF_HEIGHT, NNF_WIDTH

(167, 231)

In [44]:
a.shape

(167, 231, 2)

In [45]:
output_nnf.astype(np.int)

array([[[ 10,   2,   3],
        [ 10,   3,   2],
        [  9,   8,   0],
        ...,
        [ 19, 210,   0],
        [ 19, 211,   0],
        [ 19, 206,   0]],

       [[  7,  10,   0],
        [  7,  11,   0],
        [  7,  12,   0],
        ...,
        [ 20, 210,   0],
        [ 20, 211,   0],
        [ 20, 206,   0]],

       [[  8,  10,   0],
        [  8,  11,   0],
        [  8,  12,   0],
        ...,
        [ 21, 194,   0],
        [ 21, 195,   0],
        [ 21, 206,   0]],

       ...,

       [[153,   2,   0],
        [163,  14,   0],
        [164,  12,   0],
        ...,
        [ 93, 213,   0],
        [ 93, 214,   0],
        [ 93, 215,   0]],

       [[145,  11,   0],
        [164,  14,   0],
        [165,  12,   0],
        ...,
        [ 94, 213,   0],
        [ 94, 214,   0],
        [ 94, 215,   0]],

       [[146,  11,   0],
        [165,  14,   0],
        [166,  12,   0],
        ...,
        [ 95, 213,   0],
        [ 95, 214,   0],
        [ 95, 215,   0]]

In [46]:
np.min(lib_offsets)

-100.0

In [47]:
nnf

array([[[1.00000000e+01, 2.00000000e+00, 3.03730873e+00],
        [1.00000000e+01, 3.00000000e+00, 2.00692042e+00],
        [9.00000000e+00, 8.00000000e+00, 2.00445982e-01],
        ...,
        [1.90000000e+01, 2.10000000e+02, 3.12341407e-02],
        [1.90000000e+01, 2.11000000e+02, 3.30180700e-02],
        [1.90000000e+01, 2.06000000e+02, 2.32218378e-02]],

       [[7.00000000e+00, 1.00000000e+01, 1.62106882e-01],
        [7.00000000e+00, 1.10000000e+01, 2.01768551e-02],
        [7.00000000e+00, 1.20000000e+01, 1.12402922e-01],
        ...,
        [2.00000000e+01, 2.10000000e+02, 3.89542484e-02],
        [2.00000000e+01, 2.11000000e+02, 4.03383314e-02],
        [2.00000000e+01, 2.06000000e+02, 2.87889273e-02]],

       [[8.00000000e+00, 1.00000000e+01, 1.68289120e-01],
        [8.00000000e+00, 1.10000000e+01, 3.35409458e-02],
        [8.00000000e+00, 1.20000000e+01, 3.46251442e-01],
        ...,
        [2.10000000e+01, 1.94000000e+02, 6.42983468e-02],
        [2.10000000e+01, 1.95

In [48]:
nnf[100, 200]

array([119.        , 197.        ,   2.83793925])

In [49]:
lib_offsets[100, 210]

array([15., -9.], dtype=float32)

In [50]:
imgs[0][100, 200]

Array([0.27843137, 0.21176471, 0.29019608])

In [54]:
imgs[1][119, 197]

Array([0.18823529, 0.12156863, 0.19215686])

In [52]:
imgs[1][115, 201]

Array([0.56470588, 0.49411765, 0.43137255])

In [53]:
imgs[0].shape

(175, 239, 3)